In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 국내 최고의 투자 자문 AI 입니다."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# template = """

#     {chat_history}
#     Human:{question}
#     You:
# """


# chain = LLMChain(
#     llm=llm,
#     memory=memory,
#     prompt=prompt,
#     verbose=True,
# )

# chain.predict(question="2024-08-02, 오늘의 시황")


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


invoke_chain("2024-08-02, 오늘의 시황")

content='안녕하세요. 2024년 8월 2일의 시장은 어제 미국의 경제지표 발표와 미중 무역 협상 관련 소식에 영향을 받았습니다. 미국의 경제지표가 예상보다 좋은 결과를 보여 시장에 긍정적인 영향을 주었고, 미중 무역 협상 관련 소식도 긍정적으로 평가되어 주식시장이 상승했습니다. 미국의 다우존스 지수와 S&P 500 지수는 상승했으며, 한국 증시인 코스피와 코스닥도 상승하였습니다. 전반적으로 글로벌 주식시장은 호황을 보였습니다. 이러한 상황에서는 안정적인 투자 전략을 유지하는 것이 중요할 것으로 판단됩니다. 추가적인 정보가 필요하시면 언제든지 물어보세요.'


In [3]:
invoke_chain("2024-08-02 오늘 코스피가 3%이상 하락했는데? 다시 확인해줄래?")

content='죄송합니다. 제가 이전에 제공한 정보가 잘못되었습니다. 2024년 8월 2일에는 코스피가 3% 이상 상승했습니다. 미국의 긍정적인 경제 지표와 미중 무역 협상 소식으로 인해 시장이 상승했으며, 다우존스, S&P 500, 코스피, 코스닥 등의 지수가 모두 상승했습니다. 전반적으로 글로벌 주식 시장은 상승 흐름을 보였습니다. 이러한 상황에서는 안정적인 투자 전략을 유지하는 것이 좋습니다. 이전에 제공한 정보가 잘못되어 죄송합니다. 감사합니다.'


In [ ]:
chain.predict(question="오늘의 국내 증시는 어땠니?")

In [ ]:
memory.load_memory_variables({})